In [1]:
!pip install -U ckiptagger

In [2]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:39<00:00, 47.0MB/s]


In [3]:
!pwd

/content


In [4]:
with open("data.txt", "r", encoding = "utf-8") as f:
    data = f.read()

print(data)

拜登和習近平將在舊金山會面，討論如何避免核戰爭、人工智慧殺人機器和台灣問題。這是一場歷史性的峰會，因為這是兩位領導人第一次面對面交流，而不是通過翻譯或Zoom。拜登希望能夠說服習近平，中國不應該干涉美國的內政，比如支持俄羅斯、哈馬斯和菲律賓。習近平則希望能夠說服拜登，美國不應該干涉中國的內政，比如支持台灣、維吾爾和香港。兩位領導人都有自己的底線，但也有共同的利益，比如維持貿易、防止疫情和打擊芬太尼。這次會談的成果將取決於兩國是否能夠重啟軍事交流，讓兩軍能夠直接溝通，而不是用飛機艦艇互相威脅。這是一個艱難的任務，但也是一個必要的步驟，因為如果不這樣做，世界可能會變成一個更危險的地方。當然，這也可能是一個無聊的會談，因為兩國在很多問題上都沒有太多妥協空間，而且會談後也不會發布聯合公報。所以，我們只能祈禱，這次會談不會變成一場空洞的禮節性活動，而是一場真正的對話，能夠為中美關係帶來一些改善。不過，我們也不要抱太大的期望，因為這是拜登和習近平，而不是奧巴馬和達賴喇嘛。


In [5]:
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [9]:
result = []
i = data
line = []
line.append(i)
# print(line)
seg_res = ws(line)
pos_res = pos(seg_res)
ner_res = ner(seg_res, pos_res)
# print(ner_res)
keywords = []
skip = ['CARDINAL', 'DATE', 'TIME', 'MONEY', 'ORDINAL', 'QUANTITY', 'PERCENT']
if len(ner_res) != 0:
  for i in ner_res[0]:
    # print(i[3])
    if i[2] in skip: continue
    keywords.append({'content': i[3],'type': i[2], 'head': i[0]})

idx = 0
for i in range(len(seg_res[0])):
  # print(pos_res[0][i], seg_res[0][i])
  if pos_res[0][i] == 'Na':
    if i != 0 and pos_res[0][i-1] == 'Na': # merging noun as they are likely to be refering a single object
      keywords[-1]['content'] += seg_res[0][i]
    elif i >= 2 and pos_res[0][i-1] == 'DE' and pos_res[0][i-2] == 'VH': # to include adj+noun (ex. 無聊的會議)
      keywords.append({'content': seg_res[0][i-2]+seg_res[0][i], 'type': 'VH+Na', \
                        'head': (idx-len(seg_res[0][i-1])-len(seg_res[0][i-2]))})
    else:
      keywords.append({'content': seg_res[0][i], 'type': 'Na', 'head': idx})
  # if pos_res[0][i] == 'Nb' and seg_res[0][i] not in keywords:
  #     keywords.append({'content': seg_res[0][i], 'type': 'Nb', 'head': idx})
  idx += len(seg_res[0][i])

idx = 0
if len(keywords) == 0:
  for i in range(len(seg_res[0])):
    if 'V' in pos_res[0][i] and pos_res[0][i] != 'VH':
      if i >= 2 and pos_res[0][i-1] == 'DE' and pos_res[0][i-2] == 'VH':
        keywords.append({'content': seg_res[0][i-2]+seg_res[0][i], 'type': 'VH+V', \
                        'head': (idx-len(seg_res[0][i-1])-len(seg_res[0][i-2]))})
      else:
        keywords.append({'content': seg_res[0][i], 'type': 'V', 'head': idx})
    idx += len(seg_res[0][i])

print(keywords)
# result.append(keywords)

[{'content': '菲律賓', 'type': 'GPE', 'head': 117}, {'content': '拜登', 'type': 'PERSON', 'head': 80}, {'content': '美國', 'type': 'GPE', 'head': 99}, {'content': '美', 'type': 'GPE', 'head': 388}, {'content': '中國', 'type': 'GPE', 'head': 141}, {'content': '達賴喇嘛', 'type': 'PERSON', 'head': 431}, {'content': '習近平', 'type': 'PERSON', 'head': 420}, {'content': '美國', 'type': 'GPE', 'head': 134}, {'content': '哈馬斯', 'type': 'GPE', 'head': 113}, {'content': '香港', 'type': 'GPE', 'head': 158}, {'content': '拜登', 'type': 'PERSON', 'head': 0}, {'content': '台灣', 'type': 'GPE', 'head': 33}, {'content': '舊金山', 'type': 'GPE', 'head': 8}, {'content': '拜登', 'type': 'PERSON', 'head': 131}, {'content': '奧巴馬', 'type': 'PERSON', 'head': 427}, {'content': '中', 'type': 'GPE', 'head': 387}, {'content': '台灣', 'type': 'GPE', 'head': 151}, {'content': '中國', 'type': 'GPE', 'head': 92}, {'content': '俄羅斯', 'type': 'GPE', 'head': 109}, {'content': '維吾爾', 'type': 'GPE', 'head': 154}, {'content': '拜登', 'type': 'PERSON', 'head'

In [13]:
to_delete = []
for i in keywords:
  for j in keywords:
    if i["head"] < j["head"] and i["head"] + len(i["content"]) >= (j["head"]+len(j["content"])):
      print(i, j)
      to_delete.append(j)

keywords.remove(j)

sorted_keywords = sorted(keywords, key=lambda x: int(x["head"]))
print(sorted_keywords)

{'content': '達賴喇嘛', 'type': 'PERSON', 'head': 431} {'content': '喇嘛', 'type': 'Na', 'head': 433}
[{'content': '拜登', 'type': 'PERSON', 'head': 0}, {'content': '習近平將', 'type': 'PERSON', 'head': 3}, {'content': '舊金山', 'type': 'GPE', 'head': 8}, {'content': '核戰爭', 'type': 'Na', 'head': 20}, {'content': '人工智慧', 'type': 'Na', 'head': 24}, {'content': '機器', 'type': 'Na', 'head': 30}, {'content': '台灣', 'type': 'GPE', 'head': 33}, {'content': '問題', 'type': 'Na', 'head': 35}, {'content': '歷史性', 'type': 'Na', 'head': 42}, {'content': '峰會', 'type': 'Na', 'head': 46}, {'content': '領導人', 'type': 'Na', 'head': 55}, {'content': '翻譯', 'type': 'Na', 'head': 72}, {'content': 'Zoom', 'type': 'PERSON', 'head': 75}, {'content': '拜登', 'type': 'PERSON', 'head': 80}, {'content': '中國', 'type': 'GPE', 'head': 92}, {'content': '美國', 'type': 'GPE', 'head': 99}, {'content': '內政', 'type': 'Na', 'head': 102}, {'content': '俄羅斯', 'type': 'GPE', 'head': 109}, {'content': '哈馬斯', 'type': 'GPE', 'head': 113}, {'content': '菲

In [ ]:
text = ["我們只能祈禱"]
word_seg = ws(text) #斷句
print(word_seg)
pos_result = pos(word_seg)
print(pos_result)
ner_result = ner(word_seg, pos_result)
print(ner_result)


[['我們', '只', '能', '祈禱']]
[['Nh', 'Da', 'D', 'VE']]
[set()]


In [ ]:
for i in range(len(word_seg[0])):
  print(word_seg[0][i], pos_result[0][i])

我們 Nh
只 Da
能 D
祈禱 VE
